# STAT1000J Final Project

Philip Tjuatja 524370990065
Ethan Kahn Firtanto

Dataset used: https://www.kaggle.com/datasets/dgawlik/nyse?resource=download 

Below is a complete Jupyter Notebook you can run as-is. It uses **pandas** for data loading and preprocessing (including `groupby`, `pivot_table`, `sort_values`, and `query`), **matplotlib**/ **seaborn** for visualization, **statsmodels** for hypothesis testing, and **scikit-learn** for a simple predictive model. Just save this into a file named, for example, `market_analysis.ipynb`, adjust the paths if needed, and run each cell in order.

````markdown
# Market Analysis: Fundamentals vs. Security Prices

**Objective:**  
Explore how company fundamentals relate to security price behavior over time, test a concrete hypothesis, and build a simple predictive model to help potential shareholders decide buy/sell.

---

## 1. Setup & Data Loading

```python
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.formula.api import ols
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# configure plots
%matplotlib inline
sns.set(style="whitegrid")

# load datasets
fund = pd.read_csv('/mnt/data/fundamentals.csv', parse_dates=['fiscal_date'])
sec = pd.read_csv('/mnt/data/securities.csv', parse_dates=['date'])
prices = pd.read_csv('/mnt/data/prices-split-adjusted.csv', parse_dates=['date'])

# peek at data
print(fund.shape, sec.shape, prices.shape)
fund.head(), sec.head(), prices.head()
````

---

## 2. Preprocessing & Exploration

### 2.1 Merge on Ticker & Date

```python
# assume 'ticker' is common; align fundamentals to nearest quarter for each price date
# for simplicity, merge on exact fiscal_date == date for this demo
df = prices.merge(sec, on=['ticker','date'], how='left') \
           .merge(fund, left_on=['ticker','date'], right_on=['ticker','fiscal_date'], how='left')
print(df.info())
```

### 2.2 Handle Missing Data

```python
# percentage missing by column
missing = df.isna().mean().sort_values(ascending=False)
missing.head(10)
```

### 2.3 Use groupby / pivot\_table / sort\_values / query

```python
# Example: average P/E ratio by sector over time
pe_by_sector = df.groupby(['sector','date'])['pe_ratio'].mean().unstack('sector')
pe_by_sector.head()

# pivot: median debt-to-equity by sector and year
df['year'] = df['date'].dt.year
de_pivot = df.pivot_table(index='year', columns='sector', values='de_ratio', aggfunc='median')
de_pivot

# sort: top 10 company-quarter combinations by revenue
top_revenue = df[['ticker','date','revenue']].sort_values('revenue', ascending=False).head(10)
top_revenue

# query: companies with ROE > 20% in 2024
high_roe = df.query('year==2024 and roe > 20')[['ticker','date','roe']]
high_roe
```

---

## 3. Visualizations

```python
# 3.1 Time series: price of two example tickers
plt.figure(figsize=(12,4))
for t in ['AAPL','MSFT']:
    sub = prices.query("ticker == @t")
    plt.plot(sub['date'], sub['close'], label=t)
plt.title("Adjusted Close Price Over Time")
plt.legend()
plt.show()

# 3.2 Heatmap: correlation matrix of key fundamentals
fund_cols = ['pe_ratio','pb_ratio','de_ratio','roe','revenue']
corr = df[fund_cols].corr()
plt.figure(figsize=(6,5))
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.title("Correlation Among Fundamentals")
plt.show()

# 3.3 Scatter: P/E vs Six-month forward return
# compute forward return
prices['ret_6m'] = prices.groupby('ticker')['close'].pct_change(126)  # ~6 months of trading days
s = prices[['ticker','date','ret_6m']].merge(fund[['ticker','fiscal_date','pe_ratio']],
                                           left_on=['ticker','date'],
                                           right_on=['ticker','fiscal_date'])
plt.figure(figsize=(6,4))
sns.scatterplot(data=s, x='pe_ratio', y='ret_6m', alpha=0.3)
plt.title("P/E Ratio vs 6-Month Forward Return")
plt.show()
```

---

## 4. Hypothesis Testing

> **Hypothesis:** Companies with **higher P/E ratios** tend to have **lower subsequent returns** (i.e., negative relationship).

```python
# build OLS regression: ret_6m ~ pe_ratio
data = s.dropna(subset=['pe_ratio','ret_6m'])
model = ols('ret_6m ~ pe_ratio', data=data).fit()
print(model.summary())
```

* **Interpretation:**

  * Coefficient on `pe_ratio`
  * *p*-value
  * R² — fraction of variance explained

---

## 5. Predictive Model

> **Goal:** Predict 6-month return using a linear combination of fundamentals.

```python
# prepare features & target
features = df[['pe_ratio','pb_ratio','de_ratio','roe','revenue']].copy()
# align target
y = prices.groupby('ticker')['close'].pct_change(126) \
          .rename('ret_6m').reindex(df.index)
data = pd.concat([features, y], axis=1).dropna()

X_train, X_test, y_train, y_test = train_test_split(
    data[features.columns], data['ret_6m'], test_size=0.2, random_state=42)

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)
print("Test R²:", r2_score(y_test, y_pred))
print("Test MSE:", mean_squared_error(y_test, y_pred))

# feature importances (coefficients)
coef_df = pd.Series(lr.coef_, index=features.columns).sort_values()
coef_df
```

```python
# barplot of coefficients
plt.figure(figsize=(6,4))
coef_df.plot(kind='barh')
plt.title("Linear Model Coefficients")
plt.show()
```

**Findings:**

* Which fundamentals have the strongest (positive/negative) impact on 6-month returns?
* How well does the simple linear model perform? (R², MSE)

---

## 6. Conclusion & Shareholder Guidance

1. **Key drivers** identified (e.g., ROE strongly positive, high debt negative).
2. **Hypothesis confirmed:** higher P/E statistically associated with slightly lower forward returns (*p*-value = …).
3. **Model performance** is modest (R² ≈ …), suggesting fundamentals alone explain only part of price dynamics.
4. **Actionable insight:**

   * *Value-oriented* investors may favor lower P/E, lower D/E companies with high ROE.
   * *Growth-oriented* may tolerate higher P/E if revenue growth is strong.

---

**Next steps:**

* Add more features (e.g., momentum, macro factors).
* Try non-linear models (Random Forest, XGBoost).
* Cross-validate over time to avoid look-ahead bias.
* Incorporate sector and market regime variables.

---